In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold

In [56]:
df = pd.read_csv("../dataset/hotels_train_procesado.csv")
df.columns

Index(['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date', 'id', 'is_canceled', 'hotel_encoding',
       'tiene_hijos', 'es_extranjero', 'es_portugues', 'es_ingles',
       'es_frances', 'es_espanol', 'es_aleman', 'reserved_room_was_assigned',
       'required_a_car_parking_space', 'solo_desayuna_en_el_hotel',
       'pidio_todas_comidas', 'reservo_online', 'reservo_en_grupo',
       'reservo_sin_reembols

In [57]:
target = "is_canceled"

attrs = [
    "hotel_encoding",
    
    "lead_time",
    # "reservo_el_mismo_dia",
    "adr",

    "stays_in_weekend_nights",
    "stays_in_week_nights",

    "tiene_hijos",
    "adults",
    # "children",
    # "babies",

    # Fechas
    "arrival_date_week_number",
    "arrival_date_day_of_month",
    "arrival_date_year",

    # "es_extranjero",
    "es_portugues",
    "es_ingles",
    "es_frances",
    "es_espanol",
    "es_aleman",

    "reserved_room_was_assigned",

    "is_repeated_guest",
    "previous_cancellations", "previous_bookings_not_canceled",
    "total_of_special_requests",
    "booking_changes",

    # 'meal',
    "solo_desayuna_en_el_hotel",
    "pidio_todas_comidas",

    # 'required_car_parking_spaces',
    "required_a_car_parking_space",

    "reservo_sin_reembolso",
    "reservo_sin_depositar",
    "reservo_online",
    "reservo_en_grupo",
    "reservo_sin_compania",

    # "days_in_waiting_list",
    "tuvo_dias_de_espera",
    
     "agent9",
    "agent240",
    "agent1",
    "agent1",
    "agent6",
]

In [58]:
X = df[attrs].values
y = df[target].values

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.2)

In [59]:
params_grid = {
    "max_depth": list(range(10, 30)), 
    "min_samples_leaf": list(range(15, 35)), 
    "min_samples_split": list(range(20, 50)),
    "ccp_alpha": np.linspace(0, 2.5e-4, 50),
}

kfoldcv = StratifiedKFold(n_splits=3)

tree = DecisionTreeClassifier(criterion="gini")

model = RandomizedSearchCV(
    estimator=tree,
    param_distributions=params_grid,
    scoring=make_scorer(f1_score),
    cv=kfoldcv,
    n_iter=20,
)

In [60]:
model.fit(X_train, y_train)
print("Los mejores parametros encontrados son: ", model.best_params_)
print("con un f1 score: ", model.best_score_)

Los mejores parametros encontrados son:  {'min_samples_split': 44, 'min_samples_leaf': 15, 'max_depth': 26, 'ccp_alpha': 9.693877551020408e-05}
con un f1 score:  0.8496248269601708


In [35]:
model.score(X_test, y_test)

0.8495673573072954

In [61]:
tree = tree.set_params(**model.best_params_)
tree.fit(X_train, y_train)

tree.score(X_test, y_test)

0.8565775660179278

In [62]:
plt.figure(figsize=(100,100))
tree_plot = plot_tree(tree,
                         max_depth=1,
                         feature_names=df[attrs].columns.to_list(),
                         filled=True,
                         rounded=True,
                         class_names=True)
plt.show()

In [64]:
df_test = pd.read_csv("../dataset/hotels_test_procesado.csv")
assert(len(df_test)==26535)

prediccion = model.predict(df_test[attrs].values)
assert(len(prediccion)==26535)

submit = pd.DataFrame(
    {"is_canceled": prediccion},
    index=df_test["id"],
)
assert(len(submit)==26535)

submit.to_csv("../dataset/submit.csv")